In [1]:
# data analysis with pandas

In [2]:
## Join, Combining and Reshap Data Frames 

In [3]:
### 8.1 Hierarchical Indexing

In [79]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(10),
        index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd', 'a'],
        [1, 2, 3, 1, 3, 1, 2, 2, 3,1] ] )
display(data)
#hierarchically indexed object partial indexing is possible


a  1    1.310147
   2   -0.240333
   3   -0.264407
b  1   -1.484646
   3   -2.038001
c  1    1.825248
   2    0.857873
d  2   -0.446562
   3   -0.902258
a  1    0.238670
dtype: float64

In [80]:
display( data['b'] )
#data['b':'c']
#data.loc[['b', 'd']]

#Selection is even possible from an “inner” level:
#data.loc[:, 2]


1   -1.484646
3   -2.038001
dtype: float64

In [81]:
data['b':'c']

UnsortedIndexError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

In [60]:
data.loc[['b', 'd']]

b  1   -1.366834
   3   -0.220879
d  2   -1.372406
   3   -1.858761
dtype: float64

In [61]:
data.loc[:, 2]

a   -0.205146
c   -1.458498
d   -1.372406
dtype: float64

In [68]:
#Hierarchical indexing plays an important role in reshaping data and group-based
#operations like forming a pivot table. For example, 
#you could rearrange the data into
#a DataFrame using its unstack method:
df =  data.unstack() 
display(df)
df.stack()

,1,2,3
a,-0.256412,-0.205146,0.214023
b,-1.366834,NaN,-0.220879
c,1.095693,-1.458498,NaN
d,NaN,-1.372406,-1.858761


a  1   -0.256412
   2   -0.205146
   3    0.214023
b  1   -1.366834
   3   -0.220879
c  1    1.095693
   2   -1.458498
d  2   -1.372406
   3   -1.858761
dtype: float64

In [13]:
data.unstack().stack()

a  1   -1.360416
   2    1.526774
   3   -0.056410
b  1   -0.359090
   3   -1.970304
c  1   -0.367892
   2   -0.995830
d  2    1.294485
   3    0.133968
dtype: float64

In [15]:
#With a DataFrame, either axis can have a hierarchical index
# [18]
import pandas as pd
import numpy as np
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
        index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
        columns=[['Ohio', 'Ohio', 'Colorado'],
        ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
display(frame)
#frame['Ohio']

#Be careful to distinguish the index names 'state' and 'color'
#from the row labels

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
#@@  Reordering and sorting Levels
frame.swaplevel('key1', 'key2')
#can save in the same frame variables 
# or another variables like
# frame = frame.swaplevel('key1', 'key2')


In [ ]:
#sort_index , on the other hand, sorts the data using only the values in a single level.
#When swapping levels, it’s not uncommon to also use sort_index so that the result is
#lexicographically sorted by the indicated level:
frame.sort_index(level=1)
# compare the above statement output with following statement output
frame.swaplevel(0, 1).sort_index(level=0)

### Summary Statistics by Level
aggregate by level on either the rows or columns

In [20]:
display(frame)
frame.sum(level='key2')

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [21]:
display(frame)

frame.sum(level='color', axis=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [23]:
# Indexing with a DataFrame’s columns
frame = pd.DataFrame({'a': range(7), 
                      'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]}
                    )
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [24]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [25]:
#By default the columns are removed from the DataFrame, 
#though you can leave them in:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [26]:
#reset_index , on the other hand, does the opposite of set_index ; the hierarchical
#index levels are moved into the columns
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### Combining and Merging Datasets
* pandas.merge connects rows in DataFrames based on one or more keys.
* pandas.concat concatenates or “stacks” together objects along an axis
* The combine_first instance method enables splicing together overlapping data  to fill in missing values in one object with values from another.

In [95]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})
pd.merge(df1, df2, on='key')
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
        'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [31]:
df1 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})
pd.merge(df1, df2, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [ ]:
# remmebr to import libararies if required

left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)} )
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

pd.merge(left1, right1, left_on='key', right_index=True)
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')
pd.merge(left1, right1, left_on='key', right_index=True)

### Concatenating Along an Axis
* Another kind of data combination operation is referred to interchangeably as concat‐
enation, binding, or stacking

In [ ]:
#In [82]: s1 = pd.Series([0, 1], index=['a', 'b'])
#In [83]: s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
# In [84]: s3 = pd.Series([5, 6], index=['f', 'g'])
#pd.concat([s1, s2, s3])
#By default concat works along axis=0
#pd.concat([s1, s2, s3], axis=1)
#In this case there is no overlap on the other axis, which as you can see is the sorted
#union (the 'outer' join) of the indexes. You can instead intersect them by passing
#join='inner' :


In [71]:
#In [87]: s4 = pd.concat([s1, s3])
#In [89]: pd.concat([s1, s4], axis=1)
#In [90]: pd.concat([s1, s4], axis=1, join='inner')

#In this last example, the 'f' and 'g' labels disappeared because of the join='inner'
#option

#You can even specify the axes to be used on the other axes with join_axes :
#In [91]: pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

In [ ]:
#In [82]: s1 = pd.Series([0, 1], index=['a', 'b'])
#In [83]: s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
# In [84]: s3 = pd.Series([5, 6], index=['f', 'g'])
#result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [ ]:
#A last consideration concerns DataFrames in which the row index 
#does not contain any relevant data:

In [75]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
display(df1)
display( df2)
pd.concat([df1, df2], ignore_index=True)
pd.concat([df1, df2], ignore_index=True, sort=True)


,a,b,c,d
0,-0.616162,-0.334748,1.830146,-0.558541
1,1.590660,0.427168,1.124861,1.867629
2,-1.588619,-0.001291,-0.891064,-1.452449


,b,d,a
0,0.981109,1.348128,0.404504
1,-0.904020,1.225098,-1.443440


,a,b,c,d
0,-0.616162,-0.334748,1.830146,-0.558541
1,1.590660,0.427168,1.124861,1.867629
2,-1.588619,-0.001291,-0.891064,-1.452449
3,0.404504,0.981109,NaN,1.348128
4,-1.443440,-0.904020,NaN,1.225098


In [88]:
### Pivoting “Long” to “Wide” Format
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [89]:
periods = pd.PeriodIndex(year=data.year, 
                         quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


### Pivoting “Wide” to “Long” Format
* An inverse operation to pivot for DataFrames is pandas.melt . Rather than transforming one column into many in a new DataFrame, it merges multiple columns into one, producing a DataFrame that is longer than the input.


In [92]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]})
display(df)
melted = pd.melt(df, ['key'])
melted

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9
